In [1]:
import pandas as pd

FAOSTAT_country_data = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/FAOSTAT_countries.csv", engine='python')
FAOSTAT_country_groups = FAOSTAT_country_data["Country Group"].unique()
FAOSTAT_countries = FAOSTAT_country_data["Country"].unique()
CONTINENTS = ['Africa', 'Asia', 'Oceania', 'Europe', 'Northern America', 'South America', 'Antarctic Region']
PLOT_PROD = ['Roundwood', 'Sawnwood', 'Paper and paperboard', 'Wood fuel']
PRIM_PROD = ['Roundwood', 'Sawnwood']


In [2]:
forest_trade = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Forestry_Trade_Flows_E_All_Data_cleared.csv", engine='python')


In [3]:
trade_all = forest_trade[forest_trade.Element.str.match("Export Value") & forest_trade.Item.str.startswith("Forest products")][["Reporter Countries", "Partner Countries", "Item", "Year", "Unit", "Value"]].rename(columns={"Reporter Countries": "Exporting Country", "Partner Countries": "Importing Country"})


In [4]:
trade_all = trade_all[trade_all["Exporting Country"].isin(FAOSTAT_countries) & trade_all["Importing Country"].isin(FAOSTAT_countries)]

In [5]:
exports = trade_all.groupby(["Exporting Country", "Year"]).sum().reset_index()
imports = trade_all.groupby(["Importing Country", "Year"]).sum().reset_index()

export_net = pd.merge(exports, imports, how='inner', left_on=['Exporting Country','Year'], right_on = ['Importing Country','Year'], suffixes=("_Export", "_Import"))

export_net["Net_export"] = export_net["Value_Export"] - export_net["Value_Import"]
export_net = export_net[["Exporting Country", "Year", "Net_export"]].rename(columns={"Exporting Country": "Country"})

In [6]:
top_net_select = ['Canada', 'Sweden', 'Finland', 'Brazil', 'Russian Federation',
                  'Indonesia', 'Chile', 'Austria', 'Malaysia', 'New Zealand',
                  'Portugal', 'Latvia', 'Uruguay', 'Germany', 'Romania', 'Czechia',
                  'Cameroon', 'Slovakia', "Lao People's Democratic Republic", 'Norway',
                  'Gabon', 'Cambodia', 'Papua New Guinea', 'Solomon Islands', 'Congo',
                  'Myanmar', 'Ghana', 'South Africa',
                  'Suriname', 'Central African Republic',
                  'Democratic Republic of the Congo', 'Guyana', 'Equatorial Guinea',
                  'Malawi', 'United Republic of Tanzania', 'Fiji', 'Djibouti',
                  'Benin', 'Liberia', 'Timor-Leste', 'Guinea-Bissau', 'Chad',
                  'Guinea', 'Gambia', 'Dominica', 'Burundi', 'Mauritania',
                  "Democratic People's Republic of Korea", 'Tonga', 'Mali',
                  'Vanuatu', 'Bhutan', 'Burkina Faso', 'Cabo Verde', 'Somalia',
                  'Saint Vincent and the Grenadines', 'Sierra Leone', 'Belize',
                  'Togo', 'Samoa', 'Lithuania', 'Niger', 'Brunei Darussalam',
                  'Rwanda', 'Madagascar', 'French Polynesia']

In [7]:
top_net_producers = export_net[export_net.Year == 2015].sort_values(by="Net_export", ascending=False).head(75).Country.values
export_net = export_net[export_net["Country"].isin(top_net_producers)][["Country", "Year","Net_export"]]
export_net['Year'] = export_net['Year'].astype(str)

In [8]:
gdp = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/other/gdp_per_capita.csv", skiprows=3, engine='python')

gdp.loc[gdp["Country Name"].str.startswith("Czech Republic"), "Country Name"]= "Czechia"
gdp.loc[gdp["Country Name"].str.startswith("Congo, Rep"), "Country Name"] = "Congo"
gdp.loc[gdp["Country Name"].str.startswith("Congo, Dem. Rep."), "Country Name"] = "Democratic Republic of the Congo"
gdp.loc[gdp["Country Name"].str.startswith("Lao PDR"), "Country Name"] = "Lao People's Democratic Republic"
gdp.loc[gdp["Country Name"].str.startswith("Slovak Republic"), "Country Name"] = "Slovakia"

gdp = gdp.melt(id_vars=["Country Name", "Country Code", "Indicator Name",	"Indicator Code"], 
                var_name="Year", 
                value_name="GDP")

In [9]:
gdp = gdp[gdp["Country Name"].isin(top_net_producers)][["Country Name", "Year", "GDP"]]
gdp['Year'] = gdp['Year'].astype(str)

gdp_net = pd.merge(export_net, gdp, how='inner', left_on=["Country", "Year"], right_on=["Country Name", "Year"], suffixes=("_net", "_gdp"))
gdp_net = gdp_net[["Country", "Year", "Net_export", "GDP"]]


In [11]:
land = pd.read_csv("/home/martin/Documents/school/EPFL/ws/ada/disappearing_forests/data/faostat/Inputs_LandUse_E_All_Data_cleared.csv",engine='python')

plant_for = land[land.Item.isin(['Planted Forest']) & land.Area.isin(top_net_producers)][["Area", "Year", "Value"]]

area = land[land.Item.isin(['Land area']) & land.Area.isin(top_net_producers)][["Area", "Year", "Value"]]

In [12]:
fores_area = pd.merge(plant_for, area, how="left", left_on=["Area", "Year"], right_on=["Area", "Year"], suffixes=("_pl_for", "_land_ar"))[["Area", "Year", "Value_pl_for", "Value_land_ar"]]
fores_area["pl_percentage"] = fores_area["Value_pl_for"] / fores_area["Value_land_ar"]
fores_area['Year'] = fores_area['Year'].astype(str)


In [13]:
all_data = pd.merge(gdp_net, fores_area,  how='inner', left_on=["Country", "Year"], right_on = ["Area", "Year"])
all_data = all_data[["Country", "Year", "pl_percentage", "Net_export", "GDP"]]
all_data["Year"] = all_data["Year"].astype(int)
all_data["pl_percentage"] = all_data["pl_percentage"].apply(lambda x: x*100)
all_data = all_data[all_data.Net_export > 0]

In [14]:
data_year_select = all_data[all_data.Year > 2006]
data_year_select = data_year_select[data_year_select.Year <= 2015]
data_year_select = data_year_select.rename(columns={"pl_percentage": "Planted Forest(%)", "Net_export":"Net Export(1000 USD)"})


In [16]:
import plotly.express as px

fig = px.scatter(data_year_select, x="GDP", y="Planted Forest(%)", animation_frame="Year", animation_group="Country",
        size="Net Export(1000 USD)", color="Country", size_max=40, range_x=[-3000,105059], range_y=[-2,40])

fig.update_layout(
    xaxis_title="GDP (per person)",
    yaxis_title="Planted Forest (% in the land area)",
    width=1000,
    height=700,
    showlegend=False
)

## Description
When we look at the top net expotres of forest products in the world, we can see that some of these countires have a big share of planted forests in their land. These countires tends to have highier GDP and lower density of population, what allows them to increase the wood areas by planting new trees. This observtion it's not true for every country, we can see that the artificial planting of forest doesn't take a place in countries as Norway or Canada, even though their high GDP or net export. On the other hand, there are the countires with low GDP from Asia or Africa in which is the share of planted forrests. Many of these countires have high density of population, what in combination with the wealth of the country results in the fact that they don't have resources to regenerate forests destroyed by the wood production.


In [133]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import plotly.graph_objs as go
from IPython.core.display import display, HTML

plot(fig, filename = '4_gdp_export.html')
